In [ ]:
import warnings
#warnings.filterwarnings('ignore')
import scrapy
import logging
from scrapy.crawler import CrawlerProcess
from scrapy import signals
import csv

#logging.getLogger('scrapy').propagate = False
class CatalogSpider(scrapy.Spider):

    name = "catalogdata"
    start_urls = ["http://dmoztools.net/Arts/Animation/Anime/Titles/3/3%C3%973_Eyes/",
                  "http://dmoztools.net/Computers/Software/Databases/Data_Warehousing/Consultants/",
                  "http://dmoztools.net/Health/Animal/Drugs_and_Medications/",
                  "http://dmoztools.net/Health/Animal/Veterinary_Medicine/",
                  "http://dmoztools.net/Health/Animal/Alternative_Medicine/",
                  "http://dmoztools.net/Health/Medicine/Evidence_Based_Medicine/"
                 ]
    itemlist = []

    @classmethod
    def from_crawler(cls, crawler):
        spider = super().from_crawler(crawler)
        crawler.signals.connect(spider.spider_closed, signals.spider_closed)
        return spider

    
    #csv writer
    def spider_closed(self):
        with open("dmoz.csv","w", newline="") as f:
            writer = csv.DictWriter(f,['Category','WebsiteName','WebsiteURL','WebpageTitle'])
            writer.writeheader()
            for data in self.itemlist:
                writer.writerow(data)

                
    # parser
    def parse(self, response):
        for record in response.css('.site-item'):
            items = {}
            items["Category"] = response.xpath('//title/text()').get()
            items["WebsiteName"] = record.css('.site-title::text').extract_first(default='')
            items["WebsiteURL"] = record.css('.title-and-desc a::attr(href)').extract_first(default='')
            
            if items["WebsiteURL"]:
                
                request = response.follow(items["WebsiteURL"], callback=self.get_website_title)
                request.meta['items'] = items
                yield request
            self.itemlist.append(items)
            
    # parsing child page contents
    def get_website_title(self,response):
        items = response.meta['items']
        items['WebpageTitle']=response.xpath('//title/text()').extract_first(default='').strip()
        yield items
        
        

cat_spider = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0',   
})
cat_spider.crawl(CatalogSpider)
cat_spider.start()